In [ ]:
import pandas as pd
import torch
from feature_transforms.pipeline import ColumnPipeline
from utils.registry import get_encoder

# test dataframe
df = pd.DataFrame({
    'f1' : [0.1, 0.3, 0.2, 0.9],
    'f2' : [1.0, 0.2, 0.4, 0.5],
    'city' : ['NY', 'LA', 'NY', 'SF'],
})

# preprocessing
pipe = ColumnPipeline(
    numeric = ['yeo_johnson', 'whiten'],
    categorical = ['one_hot'])

X = pipe.fit_transform(df)
print('Pipeline output shape:', X.shape)

# instantiate encoder
Encoder = get_encoder('MLPEncoder')
enc = Encoder(
    input_dim = pipe.output_dim,
    output_dim = 32, # d
    hidden_dims = [64, 64]) # default

# forward pass
emb = enc(X)
print('Embedding shape:', emb.shape) # (batch, 32)


Pipeline output shape: (4, 5)
Embedding shape: torch.Size([4, 32])
